In [1]:
import requests
import simplejson
import unidecode
import operator
import json
import sys
from IPython.core.display import display, HTML
import pandas as pd
from collections import OrderedDict
from datetime import date
import random
from ml import *

pd.set_option('display.max_colwidth', -1)
display.border = 2
df_final = pd.DataFrame.from_dict([])
category = 'MLA1071'  # juegos y juguetes
categories =  { "id": "MLA5725", "name": "Accesorios para Vehículos" }, { "id": "MLA1403", "name": "Alimentos y Bebidas" }, { "id": "MLA1071", "name": "Animales y Mascotas" }, { "id": "MLA1367", "name": "Antigüedades" }, { "id": "MLA1368", "name": "Arte y Artesanías" }, { "id": "MLA1743", "name": "Autos, Motos y Otros" }, { "id": "MLA1384", "name": "Bebés" }, { "id": "MLA1246", "name": "Belleza y Cuidado Personal" }, { "id": "MLA1039", "name": "Cámaras y Accesorios" }, { "id": "MLA1051", "name": "Celulares y Teléfonos" }, { "id": "MLA1798", "name": "Coleccionables y Hobbies" }, { "id": "MLA1648", "name": "Computación" }, { "id": "MLA1144", "name": "Consolas y Videojuegos" }, { "id": "MLA1276", "name": "Deportes y Fitness" }, { "id": "MLA5726", "name": "Electrodomésticos y Aires Ac." }, { "id": "MLA1000", "name": "Electrónica, Audio y Video" }, { "id": "MLA2547", "name": "Entradas para Eventos" }, { "id": "MLA407134", "name": "Herramientas y Construcción" }, { "id": "MLA1574", "name": "Hogar, Muebles y Jardín" }, { "id": "MLA1499", "name": "Industrias y Oficinas" }, { "id": "MLA1459", "name": "Inmuebles" }, { "id": "MLA1182", "name": "Instrumentos Musicales" }, { "id": "MLA3937", "name": "Joyas y Relojes" }, { "id": "MLA1132", "name": "Juegos y Juguetes" }, { "id": "MLA3025", "name": "Libros, Revistas y Comics" }, { "id": "MLA1168", "name": "Música, Películas y Series" }, { "id": "MLA1430", "name": "Ropa y Accesorios" }, { "id": "MLA409431", "name": "Salud y Equipamiento Médico" }, { "id": "MLA1540", "name": "Servicios" }, { "id": "MLA1953", "name": "Otras categorías"}

https://api.mercadolibre.com/sites/MLA/categories
https://api.mercadolibre.com/sites/MLA/search?q=fabrica&category=MLA1403

In [2]:

###############PARAMETERS#######################################################
file = open("palabras_3_letras.txt", "r")

# PRODUCT FROM FILE OR LIST VARIABLE
lines = file.readlines()
lines = ('ps4')

min_price_list = (1,2000,10000)#,3000,8000,16000)
max_price_list = (2000,10000,20000)#,3000,8000,16000,240000)
other_locations = "Expeleta" #FILTER LIST
percentage_max_sell = "0.05" #FILTER % BELOW MAX SOLD
sell_condition = "new" #"old" # USED o NEW
min_sell = 1 # MINIMUM SELL FOR LOCATION
#############END PARAMETERS#####################################################


for product in lines:
    product = random.choice(lines)
    for category in categories:
        print(product," category : ",category['name']) # PRODUCT NAME SEARCHED
        df_temp = display_product(product,min_price_list,max_price_list,sell_condition,percentage_max_sell,category['id'],500000)
        if (len(df_temp)>0):
            df_final = df_final.append(df_temp)
        if ( len(df_final) == 0):
            df_final = df_temp
        print ("df_final length is :",len(df_final))
        df_final.to_csv('df_final.csv')
    if (len(df_final)> 2000):
        break
   


ps4  category :  Accesorios para Vehículos
df_final length is : 0
ps4  category :  Alimentos y Bebidas
df_final length is : 0
ps4  category :  Animales y Mascotas
df_final length is : 0
ps4  category :  Antigüedades
df_final length is : 0
ps4  category :  Arte y Artesanías
df_final length is : 0
ps4  category :  Autos, Motos y Otros
df_final length is : 0
ps4  category :  Bebés
df_final length is : 0
ps4  category :  Belleza y Cuidado Personal
df_final length is : 0
ps4  category :  Cámaras y Accesorios
df_final length is : 0
ps4  category :  Celulares y Teléfonos
df_final length is : 0
ps4  category :  Coleccionables y Hobbies
df_final length is : 1
ps4  category :  Computación
df_final length is : 43
ps4  category :  Consolas y Videojuegos
df_final length is : 119
ps4  category :  Deportes y Fitness
df_final length is : 119
ps4  category :  Electrodomésticos y Aires Ac.
df_final length is : 119
ps4  category :  Electrónica, Audio y Video
df_final length is : 119
ps4  category :  Entr

In [4]:
df_final = df_final.drop_duplicates('permalink')

In [ ]:
df_final = df_final.sort_values('total',ascending=False)
df_final.iloc[:10,:]

In [6]:
len(df_final)

129

In [7]:
df_final['sold_quantity'] = df_final.apply(lambda row: retrieve_sold_from_link(row['permalink']) , axis=1)

/home/jovyan/ml.py:45: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 45 of the file /home/jovyan/ml.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(r.text)


In [8]:
df_final['total'] = df_final['price'] * df_final['sold_quantity']

In [11]:
df_temp = df_final


In [12]:
df_temp = df_final[df_final['price'] > 12000]

In [ ]:
display(HTML(df_temp.loc[:, ['title', 'price','sold_quantity','total','state','city','thumbnail','permalink','seller_id','free_shipping']].sort_values('total',ascending=False).to_html(border=1, escape=False ,formatters=dict(thumbnail=path_to_image_html,permalink=path_to_link_html))))
 